In [1]:
%load_ext autoreload
%autoreload 2
    
# Necessary/extra dependencies. 
import shutil

#customize iPython writefile so we can write variables
from IPython.core.magic import register_line_cell_magic

@register_line_cell_magic
def writetemplate(line, cell):
    with open(line, 'w') as f:
        f.write(cell.format(**globals()))

dataset = "classification/dataset_fold_cls"
!cd yolov5

In [2]:
%cd yolov5
!pwd

/home/calcio/cardboard_yolo/yolov5
/home/calcio/cardboard_yolo/yolov5


In [3]:
from utils.downloads import attempt_download
p5 = ['n', 's', 'm']
cls = [f'{x}-cls' for x in p5]
for x in cls:
    attempt_download(f'weights/yolov5{x}.pt')

['n-cls', 's-cls', 'm-cls']


In [8]:
!python3 classify/train.py --model yolov5n-cls.pt --img 640 --batch 16 --epochs 2 --data dataset_hole_cls --pretrained weights/yolov5n-cls.pt --cache

classify/train: model=yolov5n-cls.pt, data=dataset_hole_cls, epochs=2, batch_size=16, imgsz=640, nosave=False, cache=ram, device=, workers=8, project=runs/train-cls, name=exp, exist_ok=False, pretrained=weights/yolov5n-cls.pt, optimizer=Adam, lr0=0.001, decay=5e-05, label_smoothing=0.1, cutoff=None, dropout=None, verbose=False, seed=0, local_rank=-1
github: ⚠️ YOLOv5 is out of date by 8 commits. Use 'git pull' or 'git clone https://github.com/ultralytics/yolov5' to update.
YOLOv5 🚀 v7.0-326-gec331cbd Python-3.10.14 torch-2.3.1+cu121 CUDA:0 (NVIDIA GeForce RTX 4090, 24210MiB)

TensorBoard: Start with 'tensorboard --logdir runs/train-cls', view at http://localhost:6006/
ClearML: WARNING ⚠️ ClearML is installed but not configured, skipping ClearML logging. See https://docs.ultralytics.com/yolov5/tutorials/clearml_logging_integration
albumentations: ⚠️ not found, install with `pip install albumentations` (recommended)
Model summary: 149 layers, 1213281 parameters, 1213281 gradients, 3.0 GF

In [9]:
import torch
torch.cuda.is_available()

True